## Waddah Attar Explosion V2 [SHK]

<https://ru.tradingview.com/script/d9IjcYyS-Waddah-Attar-Explosion-V2-SHK/>

>Developed by [@edyatl](https://github.com/edyatl) April 2023 <edyatl@yandex.ru>

In [1]:
# Load Jupyter extension for auto correction coding style based on Black Lib
%load_ext nb_black

<IPython.core.display.Javascript object>

### Original Indicator code

```python
//
// @author LazyBear 
// List of all my indicators: 
// https://docs.google.com/document/d/15AGCufJZ8CIUvwFJ9W-IKns88gkWOKBCvByMEvm5MLo/edit?usp=sharing
//

// Modified for Crypto Market by ShayanKM

study("Waddah Attar Explosion V2 [SHK]", shorttitle="WAE [SHK]")
sensitivity = input(150, title="Sensitivity")
fastLength=input(20, title="FastEMA Length")
slowLength=input(40, title="SlowEMA Length")
channelLength=input(20, title="BB Channel Length")
mult=input(2.0, title="BB Stdev Multiplier")

DEAD_ZONE = nz(rma(tr(true),100)) * 3.7

calc_macd(source, fastLength, slowLength) =>
	fastMA = ema(source, fastLength)
	slowMA = ema(source, slowLength)
	fastMA - slowMA

calc_BBUpper(source, length, mult) => 
	basis = sma(source, length)
	dev = mult * stdev(source, length)
	basis + dev

calc_BBLower(source, length, mult) => 
	basis = sma(source, length)
	dev = mult * stdev(source, length)
	basis - dev

t1 = (calc_macd(close, fastLength, slowLength) - calc_macd(close[1], fastLength, slowLength))*sensitivity

e1 = (calc_BBUpper(close, channelLength, mult) - calc_BBLower(close, channelLength, mult))

trendUp = (t1 >= 0) ? t1 : 0
trendDown = (t1 < 0) ? (-1*t1) : 0

plot(trendUp, style=columns, linewidth=1, color=(trendUp<trendUp[1])?lime:green, transp=45, title="UpTrend")
plot(trendDown, style=columns, linewidth=1, color=(trendDown<trendDown[1])?orange:red, transp=45, title="DownTrend")
plot(e1, style=line, linewidth=2, color=#A0522D, title="ExplosionLine")
plot(DEAD_ZONE, color=blue, linewidth=1, style=cross, title="DeadZoneLine")

```

In [2]:
# Standard imports
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
import talib as tl

import os
from os import environ as env
from dotenv import load_dotenv
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager

<IPython.core.display.Javascript object>

In [3]:
# Install a pip package in the current Jupyter kernel
# import sys

# !{sys.executable} -m pip install -U python-dotenv

<IPython.core.display.Javascript object>

In [4]:
project_dotenv = os.path.join(os.path.abspath(""), ".env")
if os.path.exists(project_dotenv):
    load_dotenv(project_dotenv)

<IPython.core.display.Javascript object>

In [5]:
api_key, api_secret = env.get("ENV_API_KEY"), env.get("ENV_SECRET_KEY")
client = Client(api_key, api_secret)

<IPython.core.display.Javascript object>

In [6]:
klines = client.get_klines(symbol="ATOMUSDT", interval=Client.KLINE_INTERVAL_15MINUTE)

short_col_names = [
    "open_time",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "close_time",
    "qav",
    "num_trades",
    "taker_base_vol",
    "taker_quote_vol",
    "ignore",
]

<IPython.core.display.Javascript object>

In [7]:
data = pd.DataFrame(klines, columns=short_col_names)
data["open_time"] = pd.to_datetime(data["open_time"], unit="ms")
data["close_time"] = pd.to_datetime(data["close_time"], unit="ms")
data.tail(5)

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
495,2023-04-18 13:15:00,12.70600000,12.71300000,12.66200000,12.68100000,15443.85000000,2023-04-18 13:29:59.999,196005.00306000,650,5544.34000000,70337.99074000,0
496,2023-04-18 13:30:00,12.68000000,12.70100000,12.64100000,12.69000000,23399.40000000,2023-04-18 13:44:59.999,296660.29865000,999,10378.55000000,131605.39577000,0
497,2023-04-18 13:45:00,12.68900000,12.71900000,12.68900000,12.69500000,20402.64000000,2023-04-18 13:59:59.999,259204.92084000,739,9486.80000000,120531.26228000,0
498,2023-04-18 14:00:00,12.69400000,12.70000000,12.60700000,12.63200000,26710.72000000,2023-04-18 14:14:59.999,337503.06311000,1145,10612.79000000,134093.40307000,0
499,2023-04-18 14:15:00,12.63300000,12.64000000,12.62500000,12.63900000,3854.41000000,2023-04-18 14:29:59.999,48686.99427000,137,2086.52000000,26354.91385000,0


<IPython.core.display.Javascript object>

### Inputs

In [8]:
sensitivity: int = 150  # input(150, title='Sensitivity')
fastLength: int = 20  # input(20, title='FastEMA Length')
slowLength: int = 40  # input(40, title='SlowEMA Length')
channelLength: int = 20  # input(20, title='BB Channel Length')
mult: float = 2.0  # input(2.0, title='BB Stdev Multiplier')

<IPython.core.display.Javascript object>

In [9]:
# DEAD_ZONE = nz(ta.rma(ta.tr(true), 100)) * 3.7
# ta_tr = tl.TRANGE(data.high, data.low, data.close)
ta_rma = np.nan_to_num(tl.ATR(data.high, data.low, data.close, 100), nan=0)
DEAD_ZONE = ta_rma * 3.7

<IPython.core.display.Javascript object>

In [10]:
def calc_macd(source, fastLength, slowLength):
    fastMA = tl.EMA(source, fastLength)
    slowMA = tl.EMA(source, slowLength)
    return fastMA - slowMA

<IPython.core.display.Javascript object>

In [11]:
def calc_BBUpper(source, length, mult):
    basis = tl.SMA(source, length)
    dev = mult * tl.STDDEV(source, length)
    return basis + dev

<IPython.core.display.Javascript object>

In [12]:
def calc_BBLower(source, length, mult):
    basis = tl.SMA(source, length)
    dev = mult * tl.STDDEV(source, length)
    return basis - dev

<IPython.core.display.Javascript object>

In [13]:
close = data.close.to_numpy(dtype=np.double)
close_1 = np.roll(close, 1)
close_1[0] = 0
close[:10]

array([11.458, 11.477, 11.442, 11.453, 11.481, 11.484, 11.549, 11.559,
       11.523, 11.532])

<IPython.core.display.Javascript object>

In [14]:
# t1 = np.full_like(data.close.to_numpy(dtype=np.double), fill_value=np.nan)

t1 = (
    calc_macd(close, fastLength, slowLength)
    - calc_macd(close_1, fastLength, slowLength)
) * sensitivity

<IPython.core.display.Javascript object>

In [15]:
e1 = calc_BBUpper(close, channelLength, mult) - calc_BBLower(close, channelLength, mult)

<IPython.core.display.Javascript object>

In [16]:
trendUp = np.full_like(t1, fill_value=np.nan)
trendDown = np.full_like(t1, fill_value=np.nan)

for i in range(len(t1)):
    trendUp[i] = t1[i] if t1[i] >= 0 else 0
    trendDown[i] = t1[i] * -1 if t1[i] < 0 else 0

<IPython.core.display.Javascript object>

In [17]:
res = pd.DataFrame(
    {
        "open_time": data["open_time"],
        "UpTrend": trendUp,
        "DownTrend": trendDown,
        "ExplosionLine": e1,
        "DeadZoneLine": DEAD_ZONE,
    }
)
res.tail(20)

,open_time,UpTrend,DownTrend,ExplosionLine,DeadZoneLine
480,2023-04-18 09:30:00,0.139945,0.000000,0.211983,0.215006
481,2023-04-18 09:45:00,0.058509,0.000000,0.169293,0.213892
482,2023-04-18 10:00:00,0.000000,0.025786,0.162699,0.212234
483,2023-04-18 10:15:00,0.131753,0.000000,0.147097,0.211185
484,2023-04-18 10:30:00,0.036757,0.000000,0.137916,0.209998
485,2023-04-18 10:45:00,0.000000,0.093950,0.123629,0.209082
486,2023-04-18 11:00:00,0.093648,0.000000,0.117402,0.208434
487,2023-04-18 11:15:00,0.143167,0.000000,0.126456,0.207645
488,2023-04-18 11:30:00,0.442959,0.000000,0.154554,0.207678
489,2023-04-18 11:45:00,0.385321,0.000000,0.179370,0.207784


<IPython.core.display.Javascript object>